1. minimal data setup
(note: my data cleaning is nonexistent, so Jiatong's preprocessing will be different and results will differ — nevertheless, the format is what matters here)

In [1]:
from COS597.data_util import load_data, get_dataloader, prepare_data
from datasets import load_dataset
import torch
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
# CHANGE ME
DEBUG = True
SEED = 123

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [29]:
# logging boilerplate
# import logging
# handlers = [logging.StreamHandler()]
# logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
#                     datefmt='%m/%d/%Y %H:%M:%S',
#                     level=logging.INFO,
#                     handlers=handlers)
# logger = logging.getLogger(__name__)

# # load data
# raw_data = load_data(logger, name='newsroom', split='train', data_dir='/n/fs/nlp-mwtang/onthefly/data')
# if DEBUG:
#     data = raw_data.select([i for i in range(200)])
# else:
#     data = raw_data

12/03/2022 21:19:04 - INFO - __main__ - Loading dataset...
12/03/2022 21:19:05 - WARNING - datasets.builder - Using custom data configuration default-eaa8771618ae72bf


Generating train split:   0%|          | 0/995041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/108837 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/108862 [00:00<?, ? examples/s]

Dataset newsroom downloaded and prepared to /u/mwtang/.cache/huggingface/datasets/newsroom/default-eaa8771618ae72bf/1.0.0/0de29640d173dddeab4d626e8ee328c9170949d0613c72900876d6f5dd789a41. Subsequent calls will reuse this data.


12/03/2022 21:22:33 - INFO - __main__ - loaded 995041 data from newsroom.


In [38]:
# n_sample = 3000
# newsroom_raw_data = raw_data
# newsroom_data = newsroom_raw_data.select([random.randint(0,len(raw_data)-1) for i in range(n_sample)])

In [3]:
# load data
raw_data = load_dataset('cc_news', split='train')
if DEBUG:
    data = raw_data.select([i for i in range(200)])
else:
    data = raw_data

Reusing dataset cc_news (/u/mwtang/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6)


In [4]:
n_sample = 1000
cc_raw_data = raw_data
cc_data = cc_raw_data.select([random.randint(0,len(raw_data)-1) for i in range(n_sample)])

2. sparse retriever: bm25

In [5]:
from rank_bm25 import BM25Okapi

corpus = [x['text'] for x in cc_data]

tokenized_corpus = [re.split(' |\n', doc) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [6]:
query = cc_raw_data[120]['title']
tokenized_query = re.split(' |\n', query)
neighbors = bm25.get_top_n(tokenized_query, range(len(corpus)), n=20)
scores = bm25.get_scores(tokenized_query)

In [7]:
# todo filter out "Read the rest of the story"

In [114]:
query

'Man, 22, dies in Duluth neighborhood shooting'

In [115]:
cc_data[neighbors]['title']

['Man arrested, charged after trying to run over JPD officer with - MSNewsNow.com - Jackson, MS',
 'Nashville police capture Waffle House restaurant shooting suspect',
 'Is Kendall Jenner Dating NBA Player Ben Simmons?',
 'Two injured in Medical District shooting',
 'Sam Smith Announces New Album and Show in Detroit',
 'Woman dies after being dragged by red SUV',
 '‘One down, 216 to go’: Republican says she received death threat after shooting',
 '19-year-old injured in shooting at Arkansas apartment',
 'Police seek suspect after Toronto shooting',
 "Denver's Yes Please Has All The Sassy Goods You Didn't Know You Needed",
 'Wrexham hero rescues eight-month-old baby girl from drowning after her pram rolled into River Dee',
 'Man dies after driving car into canal in PSL - Fox29 WFLX TV, West Palm Beach, FL-news & weather',
 "Hyundai's Czech car workers get big pay rise as labour market tightens",
 'Tuba Restaurant & Grill - Turkish Flair in the Mission',
 "'At least they...",
 'Wright St

In [116]:
scores[neighbors]

array([14.9124702 , 14.03554677, 10.48296713, 10.47085267, 10.46014066,
       10.17979054,  9.77203427,  9.51050039,  9.50442443,  9.35809325,
        9.23256251,  9.16889743,  9.11537601,  8.76863461,  8.72634239,
        8.52716127,  8.44522421,  8.3577196 ,  8.34763229,  8.23027444])

3. dense retriever: SimCSE (FAIL: somehow no attempt at installing has succeeded)

In [121]:
!pip install simcse

In [131]:
# from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

OSError: Can't load config for 'princeton-nlp/unsup-simcse-bert-base-uncased'. Make sure that:

- 'princeton-nlp/unsup-simcse-bert-base-uncased' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'princeton-nlp/unsup-simcse-bert-base-uncased' is the correct path to a directory containing a config.json file



In [ ]:
model.build_index(sentences)
results = model.search("He plays guitar.")

In [ ]:
embeddings = model.encode("A woman is reading.")

In [ ]:
# prepare data
tokenizer = 
input_tensors = prepare_data(logger, tokenizer,
                             data,
                             title_prefix= "Title:",
                             content_prefix= "Content:", 
                             max_length=args.input_max_len
                             )

4. PPLM with on-the-fly discriminator via retrieved reference documents

In [8]:
# CHANGE ME
idx = 120
query = cc_raw_data[idx]['title']
actual_text = cc_raw_data[idx]['text']
n_pos_refs = 5
n_neg_refs = 2 * n_pos_refs # actually will just use enough to match the pos refs, so we provide more here
n_sents = 2

In [9]:
tokenized_query = re.split(' |\n', query)
neighbors = bm25.get_top_n(tokenized_query, range(len(corpus)), n=n_pos_refs)
# scores = bm25.get_scores(tokenized_query)
pos_refs = cc_data[neighbors]['text']

neg_refs = cc_data.select([random.randint(0,len(cc_data)-1) for i in range(n_neg_refs)])
neg_refs = [x['text'] for x in neg_refs]

In [10]:
import nltk
nltk.download('punkt')
import nltk.data

[nltk_data] Downloading package punkt to /u/mwtang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
# CHANGE ME
ref_epochs = 10
ref_batch_size = 8
ref_lr = 0.001 # learning rate
pretrained_model = 'gpt2-medium'
cached = False
no_cuda = False
save_model = False
log_interval = 10

In [95]:
import argparse
import csv
import json
import math
import numpy as np
import os
import time
import torch
import torch.nn.functional as F
import torch.optim
import torch.optim as optim
import torch.utils.data as data
from nltk.tokenize.treebank import TreebankWordDetokenizer
from torchtext import data as torchtext_data
from torchtext import datasets
from tqdm import tqdm, trange
from transformers import BertTokenizer, BertModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel

import PPLM
from PPLM.pplm_classification_head import ClassificationHead
from PPLM.run_pplm_discrim_train import *
from PPLM.run_pplm import *
import time

EPSILON = 1e-10
max_length_seq = 100

In [54]:
device = "cuda" if torch.cuda.is_available() and not no_cuda else "cpu"
add_eos_token = pretrained_model.startswith("gpt2")
    
idx2class = ["general", "in_topic"]
class2idx = {c: i for i, c in enumerate(idx2class)}

discriminator = Discriminator(
    class_size=len(idx2class),
    pretrained_model=pretrained_model,
    cached_mode=cached,
    device=device
).to(device)

In [55]:
discriminator_meta = {
    "class_size": len(idx2class),
    "embed_size": discriminator.embed_size,
    "pretrained_model": pretrained_model,
    "class_vocab": class2idx,
    "default_class": 0,
}

In [56]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [57]:
# turn docs into a list of triples (or groups of n_sents) of sentences in those docs
def get_sent_groups(refs):
    ref_X = []
    for x in refs:
        sents = sent_tokenizer.tokenize(x)
        sent_groups = [' '.join(sents[i:i+n_sents]) for i in range(0, len(sents), n_sents)]
        ref_X.extend(sent_groups)
    return ref_X

def create_dataset(pos_refs, neg_refs):
    # turn positive and negative docs into a single list of groups of sentences
    ref_X_pos = get_sent_groups(pos_refs)
    ref_X_neg = get_sent_groups(neg_refs)
    ref_X = ref_X_pos + ref_X_neg
    ref_y = [1] * len(ref_X_pos) + [0] * len(ref_X_neg)
    n_pos = len(ref_X_pos)
    
    # TODO remove this flakiness
    assert len(ref_X) >= 2*n_pos
        
    ref_X = ref_X[:2*n_pos] # balance pos and neg
    ref_y = ref_y[:2*n_pos] # balance pos and neg

    # randomize
    perm = np.random.permutation(len(ref_X))
    ref_X = np.array(ref_X)[perm]
    ref_y = np.array(ref_y)[perm]

    # convert dataset to tensors
    n_examples = len(ref_X)
    ref_X_tensor = []
    ref_y_tensor = []
    for i in range(len(ref_X)):
        seq = discriminator.tokenizer.encode(ref_X[i])
        if len(seq) >= max_length_seq:
            print("Line {} is longer than maximum length {}, as it has length {}, truncating".format(
                i, max_length_seq, len(seq)
            ))
            seq = seq[:max_length_seq]
        if add_eos_token:
            seq = [50256] + seq
        seq = torch.tensor(
            seq, device=device, dtype=torch.long
        )
    #     else:
    #         print("Line {} is longer than maximum length {}".format(
    #             i, max_length_seq
    #         ))
    #         continue
        ref_X_tensor.append(seq)
        ref_y_tensor.append(ref_y[i])

    n_examples = len(ref_X_tensor) # may be less than before
    print('final number of examples =', n_examples)
    
    return Dataset(ref_X_tensor, ref_y_tensor)

In [58]:
test_pos_refs = [actual_text]
test_neg_refs = cc_data.select([random.randint(0,len(cc_data)-1) for i in range(n_neg_refs)])
test_neg_refs = [x['text'] for x in test_neg_refs]

def get_dataloaders(batch_size=ref_batch_size):    
    train_dataset = create_dataset(pos_refs, neg_refs)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           collate_fn=collate_fn)

    test_dataset = create_dataset(test_pos_refs, test_neg_refs)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              collate_fn=collate_fn)
    return train_loader, test_loader

In [59]:
def train_discrim(discrim, train_loader, test_loader, lr=ref_lr, epochs=ref_epochs):
    start = time.time()
    optimizer = optim.Adam(discrim.parameters(), lr=lr)
    test_losses = []
    test_accuracies = []

    for epoch in range(epochs):
        start = time.time()
        print("\nEpoch", epoch + 1)

        train_epoch(
            discriminator=discrim,
            data_loader=train_loader,
            optimizer=optimizer,
            epoch=epoch,
            log_interval=log_interval,
            device=device
        )
        test_loss, test_accuracy = evaluate_performance(
            data_loader=test_loader,
            discriminator=discrim,
            device=device
        )

        end = time.time()
        print("Epoch took: {:.3f}s".format(end - start))

        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        print("\nExample prediction")
        predict(example_sentence, discrim, idx2class,
                cached=cached, device=device)

    #     if save_model:
    #         # torch.save(discriminator.state_dict(),
    #         #           "{}_discriminator_{}.pt".format(
    #         #               args.dataset, epoch + 1
    #         #               ))
    #         torch.save(discriminator.get_classifier().state_dict(),
    #                    classifier_head_fp_pattern.format(epoch + 1))

    min_loss = float("inf")
    min_loss_epoch = 0
    max_acc = 0.0
    max_acc_epoch = 0
    print("Test performance per epoch")
    print("epoch\tloss\tacc")
    for e, (loss, acc) in enumerate(zip(test_losses, test_accuracies)):
        print("{}\t{}\t{}".format(e + 1, loss, acc))
        if loss < min_loss:
            min_loss = loss
            min_loss_epoch = e + 1
        if acc > max_acc:
            max_acc = acc
            max_acc_epoch = e + 1
    print("Min loss: {} - Epoch: {}".format(min_loss, min_loss_epoch))
    print("Max acc: {} - Epoch: {}".format(max_acc, max_acc_epoch))

In [60]:
def reset_weights(discrim):
    discrim.get_classifier().mlp.reset_parameters()

In [66]:
test_pos_sents = get_sent_groups(test_pos_refs)
test_neg_sents = get_sent_groups(test_neg_refs)

def eval_discrim(discrim):
    print('discriminator predictions for ground truth:')
    for x in test_pos_sents:
        print(x)
        print(discrim.predict(x))

    print('discriminator predictions for random:')
    for x in test_neg_sents:
        print(x)
        print(discrim.predict(x))

In [62]:
# CHANGE ME (AGAIN, for easy tuning)
ref_epochs = 10
ref_batch_size = 8
ref_lr = 0.001 # learning rate

In [63]:
train_loader, test_loader = get_dataloaders(batch_size=ref_batch_size)
reset_weights(discriminator)
train_discrim(discriminator, train_loader, test_loader, lr=ref_lr, epochs=ref_epochs)

Line 18 is longer than maximum length 100, as it has length 108, truncating
Line 33 is longer than maximum length 100, as it has length 131, truncating
Line 50 is longer than maximum length 100, as it has length 107, truncating
Line 67 is longer than maximum length 100, as it has length 111, truncating
final number of examples = 84
final number of examples = 42

Epoch 1
Train Epoch: 1 [8/84 (10%)]	Loss: 1.531258
Train Epoch: 1 [84/84 (100%)]	Loss: 1.315004
Performance on test set: Average loss: 0.7843, Accuracy: 21/42 (50%)
Epoch took: 0.868s

Example prediction
Input sentence: This is incredible! I love it, this is the best chicken I have ever had.
Predictions: general: 0.2880, in_topic: 0.7120

Epoch 2
Train Epoch: 2 [8/84 (10%)]	Loss: 1.128045
Train Epoch: 2 [84/84 (100%)]	Loss: 0.427830
Performance on test set: Average loss: 0.5528, Accuracy: 26/42 (62%)
Epoch took: 0.841s

Example prediction
Input sentence: This is incredible! I love it, this is the best chicken I have ever had.
P

[0.22033634388300816, 0.7796636657805581]
discriminator predictions for random:
Posted by admin on January 2, 2017
Leave a Reply
You must be logged in to post a comment.
[0.7619882212593441, 0.2380117928647318]
Story highlights New report says United States failed to do its "homework" before airstrikes on civilians
Syrian reports say mosque hit; US says al Qaeda meeting targeted
(CNN) A new report suggests American forces failed to take necessary precautions to avoid dropping a pair of bombs on a Northern Syria mosque full of hundreds of worshipers.
Syrians said the US airstrikes hit a western Aleppo mosque on March 16 , killing at least 40 worshipers and injuring dozens more. The US said the bombings targeted a meeting hall hosting al Qaeda terrorists.
Interviews with locals and photographs and video of the building in question prove it was a well-known mosque that hosted lectures every Thursday between sunset and evening prayers, according to a new Human Rights Watch report.
Syria Ci

In [67]:
eval_discrim(discriminator)

discriminator predictions for ground truth:
Duluth police responded to a 911 call reporting shots fired at 510 E. 11th Street at about 2 p.m. Tuesday. The victim was taken from the home on a gurney, witnesses said, and transferred by Gold Cross Ambulance to a local hospital.
[0.057100428995520626, 0.9428995859396738]
Police confirmed just before 5 p.m. that the victim had died. Police remained on the scene into the evening hours, and at 8 p.m. Duluth police reported that its officers — as well as several agents from the federal Bureau of Alcohol, Tobacco, Firearms and Explosives — were executing search warrants in connection with the case.
[0.0811589175165375, 0.9188410401382061]
A "person of interest" had been detained but no arrests had been made as of 8 p.m., Duluth police said, with the investigation expected to continue overnight. Mayor Emily Larson said in a statement Tuesday night that she wanted to assure the Duluth community that the police department is "fully engaged" and ha

[0.9153875673195989, 0.08461248933802615]
This time the goal will be ring number six of the Tom Brady – Bill Belichick era. The Eagles outclassed the Minnesota Vikings 38-7 in the NFC Championship game to deny the Vikings the opportunity of playing in a home Super Bowl.
[0.9200663138988711, 0.07993368525607926]
New England, by contrast, conjured up a trademark fourth-quarter comeback to recover from a 20-10 deficit, before disposing of the Jacksonville Jaguars 24-20. The Patriots unsurprisingly enter Super Bowl Sunday as slight favourites in their quest to become the first side to retain a Super Bowl title since they last did it in 2005.
[0.9461426905981044, 0.053857343180076636]
But Doug Pederson’s Eagles will be no pushovers. Most sides would have been flattened by the crushing blow of losing a starting quarterback to a season-ending injury as late as December.
[0.8850001006680118, 0.11499993945894428]
Especially when that quarterback happens to be Carson Wentz who had put together a

[0.852301337119916, 0.14769872401249756]
His best of 14:45.03, done at the Olympics last summer when he finished 4th, is nearly 20 seconds clear of the next fastest active American. However, after doing both pool and open water in Rio, he has decided to take a step back from the pool for the year.
[0.9009592331220994, 0.09904074862246436]
He told Swimming World:
“I decided to take off pool Nationals,” he said. “It’s the first year of the quad, so it’s a little bit more relaxed.
[0.6923275848753915, 0.3076723813696228]
Just decided to focus on open water this summer, hopefully put up a good 10k at Worlds and then next year start focusing on the 1500 and 10k again for Pan Pacs.”
Two years ago in Kazan, when Wilimovsky was solely focusing on open water, he took the gold medal in the men’s 10K. Last summer, shortly after competing in the pool, he took 5th in a very close race.
[0.8361125855933871, 0.1638874459805924]
With Wilimovsky out, we’ve had to adjust our picks in the men’s distance 

In [83]:
discriminator.predict('Jordan Wilimovsky found it amusing that the bonobos couldnt hear')

[0.45476621984779597, 0.5452338459990632]

In [111]:
import PPLM
from PPLM.pplm_classification_head import ClassificationHead
from PPLM.run_pplm_discrim_train import *
from PPLM.run_pplm import run_pplm_example

# reload while modifying PPLM code
import imp
imp.reload(PPLM)
imp.reload(PPLM.run_pplm)

<module 'PPLM.run_pplm' from '/n/fs/nlp-mwtang/onthefly/PPLM/run_pplm.py'>

In [113]:
PPLM.run_pplm.run_pplm_example

<function PPLM.run_pplm.run_pplm_example(pretrained_model='gpt2-medium', cond_text='', uncond=False, num_samples=1, bag_of_words=None, discrim=None, discrim_weights=None, discrim_meta=None, class_label=-1, length=100, stepsize=0.02, temperature=1.0, top_k=10, sample=True, num_iterations=3, grad_length=10000, horizon_length=1, window_length=0, decay=False, gamma=1.5, gm_scale=0.9, kl_scale=0.01, seed=0, no_cuda=False, colorama=False, verbosity='regular', on_the_fly=False, classifier=None, class_id=None)>

In [123]:
prefix = ' '.join(nltk.word_tokenize(actual_text[:100])[:4])
run_pplm_example(
        pretrained_model="gpt2-medium",
        cond_text='Article Title: ' + query + ' Article Text: ' + prefix,
        uncond=False,
        num_samples=3,
        bag_of_words=None,
        discrim=None,
        discrim_weights=None,
        discrim_meta=None,
        class_label=-1,
        length=100,
        stepsize=0.02,
        temperature=1.0,
        top_k=10,
        sample=True,
        num_iterations=3,
        grad_length=10000,
        horizon_length=1,
        window_length=0,
        decay=False,
        gamma=1.5,
        gm_scale=0.9,
        kl_scale=0.01,
        seed=0,
        no_cuda=False,
        colorama=False,
        verbosity='quiet',
        on_the_fly=True, # NEW PARAMS
        classifier=discriminator.get_classifier(), # NEW PARAMS
        class_id=1, # NEW PARAMS
)

= Prefix of sentence =
<|endoftext|>Article Title: Man, 22, dies in Duluth neighborhood shooting Article Text: Duluth police responded to

= Unperturbed generated text =
<|endoftext|>Article Title: Man, 22, dies in Duluth neighborhood shooting Article Text: Duluth police responded to a reported shooting near East Main Street and Sibley Avenue in the 1200 block of East Main Street at about 3:30 a.m. Sunday. An individual was found with multiple gunshot wounds, police said. Police say the shooter was later taken into custody, but declined to give further details Sunday. Duluth police said they received a 911 call of a shooting near East Main Street and Sibley Avenue at about 3:45 a.m. Sunday. When officers arrived, they saw a

= Perturbed generated text 1 =
<|endoftext|>Article Title: Man, 22, dies in Duluth neighborhood shooting Article Text: Duluth police responded to a the victim's house in the 12-10/90000 at a 9:30:00 a in the 4a., and the, and the and of the the 2/4/4, and and the a

5. is Bag of Words PPLM better? (IN PROGRESS)

In [142]:
pos_words = []
for x in pos_refs:
    pos_words.extend(nltk.word_tokenize(x))

In [148]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder = BigramCollocationFinder.from_words(pos_words)

# only bigrams that appear 3+ times
finder.apply_freq_filter(3)

# return the 10 n-grams with the highest PMI
finder.nbest(bigram_measures.pmi, 10)

[('Las', 'Vegas'),
 ('training', 'exercise'),
 ('trip', 'wire'),
 ('what', 'we'),
 ('Reporter', ':'),
 ('That', "'s"),
 ('Kaunda', 'said'),
 ('the', 'latest'),
 (',', 'but'),
 ('.', 'A')]

In [188]:
from densephrases import DensePhrases

In [189]:
model = DensePhrases(
    load_dir='princeton-nlp/densephrases-multi-query-kilt-multi',
    dump_dir='/n/fs/nlp-mwtang/onthefly/densephrases_dump/densephrases-multi_wiki-20181220/dump',
)

This could take up to 15 mins depending on the file reading speed of HDD/SSD


usage: ipykernel_launcher.py [-h] [--seed SEED] [--draft] [--verbose_logging]
                             [--fp16] [--fp16_opt_level FP16_OPT_LEVEL]
                             [--model_type MODEL_TYPE]
                             [--pretrained_name_or_path PRETRAINED_NAME_OR_PATH]
                             [--config_name CONFIG_NAME]
                             [--tokenizer_name TOKENIZER_NAME]
                             [--load_dir LOAD_DIR] [--output_dir OUTPUT_DIR]
                             [--max_seq_length MAX_SEQ_LENGTH]
                             [--doc_stride DOC_STRIDE]
                             [--max_query_length MAX_QUERY_LENGTH]
                             [--max_answer_length MAX_ANSWER_LENGTH]
                             [--do_lower_case] [--stage STAGE]
                             [--dump_dir DUMP_DIR] [--offset OFFSET]
                             [--quantizer_path QUANTIZER_PATH]
                             [--trained_index_path TRAINED_INDEX_PAT

SystemExit: 2

In [190]:
%tb

SystemExit: 2

In [68]:
discriminator.predict('The seagulls came to the beach without seeing anything.')

[0.1872495619617495, 0.8127504624775858]

In [166]:
ref_dataset = PPLM.run_pplm_discrim_train.Dataset(ref_X, ref_y)
disc, disc_meta = train_discriminator(ref_dataset, epochs=ref_train_epochs, learning_rate=ref_lr, batch_size=ref_batch_size)

Preprocessing <PPLM.run_pplm_discrim_train.Dataset object at 0x7f1e220f4a60> dataset...


ValueError: When generic dataset is selected, dataset_fp needs to be specified aswell.